In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [37]:
df = pd.read_csv('train.csv')

- 缺失值

In [38]:
df.shape[0] - df.count()

Unnamed: 0                           0
article_contentid                    0
article_dwelltime                38603
article_l1_categories             9499
article_l2_categories             9499
article_lan                          0
article_publisher                 9521
article_title                     9499
article_update_time               9499
dt                                   0
user_age                         10962
user_all_pkgs                        0
user_brand                           0
user_country                         0
user_gender                      14335
user_gender_weight               48406
user_gp_frequency                  732
user_id                              0
user_maxmind_city                21604
user_maxmind_country_iso_code        2
user_maxmind_state_iso_code      16652
user_model                           0
dtype: int64

- load data

In [39]:
%%time
ret = []
list_key = [
    'article_contentid', 'article_dwelltime', 'article_l1_categories',
    'article_l2_categories', 'article_lan', 'article_publisher',
    'article_title', 'article_update_time', 'dt', 'user_id'
]
with open('tr_data_20170128', 'r') as f:
    for line in f:
        record_dict = {}
        record = json.loads(line)
        ret.append(dict([(key, value) for key, value in record.items() if key in list_key]))
tr = pd.DataFrame(ret)
del ret

CPU times: user 15.5 s, sys: 909 ms, total: 16.4 s
Wall time: 17.1 s


- dropna

In [40]:
tr.dropna(subset=['article_l1_categories'],how='any',inplace=True)
tr.dropna(subset=['article_l2_categories'],how='any',inplace=True)
print tr.shape[0] - tr.count()
tr.shape
dtdata = tr.pop('dt')
tr.insert(0,'dt', dtdata)

article_contentid            0
article_dwelltime        32612
article_l1_categories        0
article_l2_categories        0
article_lan                  0
article_publisher            0
article_title                0
article_update_time          0
dt                           0
user_id                      0
dtype: int64


- user dict

In [41]:
c1 = set()
c2 = set()
for i in arange(tr.shape[0]):
    for c in tr.iloc[i]['article_l1_categories']:
        c1.add(c['name'])
    for c in tr.iloc[i]['article_l2_categories']:
        c2.add(c['name'])     

In [42]:
c1, c2 = list(c1), list(c2)
df1_dict = {}
df2_dict = {}
for user in np.unique(tr['user_id']):
    df1_dict[user] = ([user] + [0.0] * len(c1))
    df2_dict[user] = ([user] + [0.0] * len(c2))
for i in arange(tr.shape[0]):
    user = tr.iloc[i]['user_id']
    for c in tr.iloc[i]['article_l1_categories']:
        name = c['name']
        weight = c['weight']
        df1_dict[user][c1.index(name)+1] += weight
    for c in tr.iloc[i]['article_l2_categories']:
        name = c['name']
        weight = c['weight']
        df2_dict[user][c2.index(name)+1] += weight

In [43]:
df1 = pd.DataFrame(df1_dict.values(),columns = ['user_id']+c1)
df1.insert(0,'dt', dtdata)
df2 = pd.DataFrame(df2_dict.values(),columns = ['user_id']+c2)
df2.insert(0,'dt', dtdata)

In [44]:
df1.to_csv('df1_articles.csv')
df2.to_csv('df2_articles.csv')

In [45]:
df1.describe()

,1000637,1001039,1000267,1000992,1000560,1001091,0010000000,1000123,6000007,1000288,...,1000395,1000721,1000661,1000374,2000000,1000111,1000031,1000298,1000931,8000001
count,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,...,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000
mean,0.422917,0.044334,0.059206,0.132941,0.330595,0.170655,0.000253,0.121375,0.137189,0.877676,...,1.793137,0.021699,3.117144,0.180200,0.000253,0.134262,4.049362,0.069901,0.295318,0.002405
std,1.867893,0.350517,0.324815,0.941094,2.035200,0.730560,0.015902,0.860023,1.322089,3.779356,...,5.625017,0.392692,16.486328,1.389499,0.022513,0.908341,12.466879,0.778649,1.577674,0.070190
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.847085,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.533624,...,1.372980,0.000000,1.584646,0.000000,0.000000,0.000000,2.839942,0.000000,0.000000,0.000000
max,45.829454,11.852662,7.468683,44.220449,65.853051,17.884988,0.999000,36.081828,67.954556,137.670987,...,135.359789,30.299777,729.557964,65.860097,2.000000,32.668289,292.801988,49.222600,43.098891,3.996000


- load data to merge

In [46]:
df = pd.read_csv('df2_articles.csv')
del df['Unnamed: 0']
dfin = pd.read_csv('Input_Data.csv')
del dfin['u\'user_id\'']
del dfin['Unnamed: 0']
print dfin.shape
print df.shape

(8031, 55)
(7892, 169)


- log fir df(y)

In [47]:
for column in df.columns:
    if ((column != 'user_id') & (column != 'dt')):
        df[column] = (df[column]-df[column].mean())/df[column].std()

In [48]:
#for column in df.columns:
#    if ((column != 'user_id') & (column != 'dt')):
#        df[column] = np.log(df[column]+1)
df.describe()

,dt,1000269,1000506,1000917,1000940,1000267,1000992,1000993,1000611,1000183,...,1000644,1000371,1000073,1000072,1000076,2000000,1000978,1000979,1001012,1000876
count,7.444000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7892.000000,...,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03,7.892000e+03
mean,2.017012e+07,-2.475916e-17,-1.260466e-17,-2.700999e-18,4.501665e-18,6.302330e-18,6.302330e-18,4.501665e-17,3.601332e-18,0.000000,...,2.700999e-17,2.700999e-18,2.363374e-17,-1.080399e-17,-3.601332e-18,3.601332e-18,-3.601332e-18,1.620599e-17,-1.890699e-17,1.012875e-17
std,5.576490e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,2.017011e+07,-1.914580e-01,-1.351456e-01,-2.210951e-02,-7.837070e-02,-9.355207e-02,-5.649777e-02,-1.445871e-01,-1.574470e-01,-0.114728,...,-1.895851e-01,-6.099471e-02,-1.727557e-01,-1.090245e-01,-1.037656e-01,-1.125658e-02,-4.309052e-02,-9.698850e-02,-2.545881e-02,-7.691591e-02
25%,2.017012e+07,-1.914580e-01,-1.351456e-01,-2.210951e-02,-7.837070e-02,-9.355207e-02,-5.649777e-02,-1.445871e-01,-1.574470e-01,-0.114728,...,-1.895851e-01,-6.099471e-02,-1.727557e-01,-1.090245e-01,-1.037656e-01,-1.125658e-02,-4.309052e-02,-9.698850e-02,-2.545881e-02,-7.691591e-02
50%,2.017012e+07,-1.914580e-01,-1.351456e-01,-2.210951e-02,-7.837070e-02,-9.355207e-02,-5.649777e-02,-1.445871e-01,-1.574470e-01,-0.114728,...,-1.895851e-01,-6.099471e-02,-1.727557e-01,-1.090245e-01,-1.037656e-01,-1.125658e-02,-4.309052e-02,-9.698850e-02,-2.545881e-02,-7.691591e-02
75%,2.017012e+07,-1.914580e-01,-1.351456e-01,-2.210951e-02,-7.837070e-02,-9.355207e-02,-5.649777e-02,-1.445871e-01,-1.574470e-01,-0.114728,...,-1.895851e-01,-6.099471e-02,-1.727557e-01,-1.090245e-01,-1.037656e-01,-1.125658e-02,-4.309052e-02,-9.698850e-02,-2.545881e-02,-7.691591e-02
max,2.017013e+07,2.441720e+01,4.123096e+01,4.989654e+01,2.745412e+01,3.001200e+01,4.993920e+01,3.487501e+01,3.594174e+01,41.734476,...,2.635835e+01,4.144829e+01,3.313772e+01,5.587159e+01,3.747520e+01,8.882567e+01,6.857855e+01,4.340575e+01,7.494881e+01,4.196470e+01


In [49]:
pd.merge(dfin,df).to_csv('merge2_std.csv',index=False)

In [50]:
# pd.read_csv('merge2_std.csv')

- scale-z_score-standerdization ***if need***